## Introduction to TensorBoard 

__TensorBoard__ is a visualization software that comes with any standard TensorFlow installation. In Google’s words: “The computations you'll use TensorFlow for - like training a massive deep neural network - can be complex and confusing. To make it easier to understand, debug, and optimize TensorFlow programs, we've included a suite of visualization tools called TensorBoard.”

TensorFlow programs can range from  very simple to super complex problems (using thousands of computations), and they all have two basic components, Operations and Tensors. As explained in the previous tutorials, the idea is that you create a model that consists of a set of operations, feed data in to the model and the tensors will flow between the operations until you get an output tensor, your result.

When fully configures, TensorBoard window will look something like this:

<img src="files/files/3_1.png" width="500" height="1000" >

___Fig1. ___ TensorBoard appearance


TensorBoard was created as a way to help you understand the flow of tensors in your model so that you can debug and optimize it. It is generally used for two main purposes:

1. Graph Visualization
2. Writing Summaries (or Visualizing Learning)

We'll cover this two main usages of TensorBoard in this tutorial. Learning to use TensorBoard early and often will make working with TensorFlow much more enjoyable and productive.

## 1. Gragh Visualizing

Will powerful, TensorFlow computation graphs can become extremely complicated. Visualizing the graph can help you understand and debug it. Here's an example of the visualization at work from TensorFlow website.

<img src="files/files/3_2.gif" width="500" height="1000" >

___Fig2. ___ Visualization of a TensorFlow graph

To make our TensorFlow program __TensorBoard-activated__, we need to add a very few lines of code to it. This will export the TensorFlow operations into a file, called __event file__ (or event log file). TensorBoard is able to read this file and give insight into the model graph and its performance.

Now let's write a simple TensorFlow program and visualize its computation graph with TensorBoard.

### Example 1:
Let's create two constants and add them together. Constant tensors can be defined simply by defining their value:


In [3]:
import tensorflow as tf

# create graph
a = tf.constant(2)
b = tf.constant(3)
c = tf.add(a, b)
# launch the graph in a session
with tf.Session() as sess:
    print(sess.run(c))

5


To visualize the program with TensorBoard, we need to write log files of the program. To write event files, we first need to create a __writer__ for those logs, using this code:

In [ ]:
writer = tf.summary.FileWriter([logdir], [graph])

where __[logdir]__ is the folder where you want to store those log files. You can choose [logdir] to be something meaningful such as './graphs'. The second argument __[graph]__ is the graph of the program we're working on. There are two ways to get the graph:
1. Call the graph using __tf.get_default_graph()__, which returns the default graph of the program
2. set it as __sess.graph__ which returns the session's graph (note that this requires us to already have created a session).

We'll show both ways in the following example; however, the second way is more common. Either way, make sure to create a writer only after you’ve defined your graph. Otherwise, the graph visualized on TensorBoard would be incomplete.

Let's add the writer to the first example and visualize the graph.


In [ ]:
import tensorflow as tf

# create graph
a = tf.constant(2)
b = tf.constant(3)
c = tf.add(a, b)

# creating the writer out of the session
# writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())

# launch the graph in a session
with tf.Session() as sess:
    # or creating the writer inside the session
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run(c))
    # don't forget to close the writer at the end
    writer.close()

Now if you run this code, it creates a directory inside your current directory (beside your Python code) which contains the __event file__.

<img src="files/files/3_3.png" width="300" height="600" >

___Fig3. ___ Created directory which contains the event file


Next, go to Terminal and make sure that the present working directory is the same as where you ran your Python code. For example, here we can switch to the directory using 

$ cd ~/Desktop/tensorboard

Then run:

$ tensorboard --logdir="./graphs" --port 6006

This will generate a link for you. ctrl+left click on that link (or simply copy it into your browser or just open your browser and go to http://localhost:6006/. This will show the TensorBoard page which will look like:

<img src="files/files/3_4.png" width="500" height="1000" >

___Fig4. ___ TensorBoard page visualizing the graph generated in Example 1

“Const” and “Const_1” in the graph correspond to a and b, and the node “Add” corresponds to c. The names we give them (a, b, and c) are just __Python-names__ which are for us to access them when we write code. They mean nothing for the internal TensorFlow. To make TensorBoard understand the names of your ops, you have to explicitly name them. 

Let's modify the code one more time and add the names:




In [ ]:
import tensorflow as tf

# create graph
a = tf.constant(2, "a")
b = tf.constant(3, "b")
c = tf.add(a, b, "addition")

# creating the writer out of the session
# writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())

# launch the graph in a session
with tf.Session() as sess:
    # or creating the writer inside the session
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run(c))
    # don't forget to close the writer at the end
    writer.close()

<img src="files/files/3_5.png" width="500" height="1000" >

___Fig5. ___ TensorBoard page visualizing the graph generated in Example 1 with modified names


__*Note:__ If you run your code several times with the same [logdir], there will be multiple event files in your [logdir]. TF will show only the latest graph and display the warning of multiple event files. To get rid of the warning, delete the event files you no longer need or save them in different [logdir] folders.


## 2. Writing Summaries to Visualize Learning

So far we only focused on how to visualize the graph in TensorBoard. In this second part, we are now going to use a special operation called __summary__ to visualize the model parameters (like weights and biases of a neural network), metrics (like loss or accuracy value), and images (like input images to a network). 

__Summary__ is a special TensorBoard operation that takes in a regular tenor and outputs the summarized data to your disk (i.e. in the event file). Basically, there are three main types of summaries:

__1. tf.summary.scalar:__ used to write a single scalar-valued tensor (like classificaion loss or accuracy value)

__2. tf.summary.histogram:__ used to plot histogram of all the values of a non-scalar tensor (like weight or bias matrices of a neural network)

__3. tf.summary.image:__ used to plot images (like input images of a network, or generated output images of an autoencoder or a GAN)

In the following sections, we'll go through each of the above summary types in more details.


### 2.1. tf.summary scalar:


